In [17]:
import numpy as np
# 4 barcos de 1 posición de eslora
# 3 barcos de 2 posiciones de eslora
# 2 barcos de 3 posiciones de eslora
# 1 barco de 4 posiciones de eslora
# ¿ como diferenciamos los barcos ?
posiciones = ([
                [(8,3),(8,4),(8,5),(8,6)]
               ,[(5,2),(6,2),(7,2)]
               ,[(2,2),(2,3),(2,4)]
               ,[(0,1), (1,1)]
               ,[(1,3), (1,4)]
               ,[(0,5), (0,6)]
               ,[(9,9)]
               ,[(0,0)]
               ,[(7,6)]
               ,[(4,2)]
              ])
def crear_tablero (dim=10):
    tablero = np.full((dim,dim)," ")
    return tablero
def colocar_barco(tablero,coordinadas):
    for i in coordinadas:
        for p in range(len(i)):
            tablero[i[p][0],i[p][1]] = "O"
    return tablero

In [18]:
tablero = crear_tablero(10)

In [24]:
def colocar_barco_plus(tablero, barco):
    tablero_temp = tablero.copy()
    filas_totales, columnas_totales = tablero.shape
    for pieza in barco:
        fila, columna = pieza
        if fila < 0  or fila >= filas_totales:
            print(f"No puedo poner la pieza {pieza} porque se sale del tablero")
            return False
        if columna < 0 or columna >= columnas_totales:
            print(f"No puedo poner la pieza {pieza} porque se sale del tablero")
            return False
        if tablero[pieza] == "O" or tablero[pieza] == "X":
            print(f"No puedo poner la pieza {pieza} porque hay otro barco")
            return False
        tablero_temp[pieza] = "O"
    return tablero_temp

In [40]:
import random 
def crea_barco_aleatorio(tablero, eslora = 4, num_intentos = 100):
    filas_totales, columnas_totales = tablero.shape
    contador = 0
    while contador <= num_intentos:
        contador += 1
        barco = []
        primera_pieza = (random.randint(0, filas_totales - 1),random.randint(0, columnas_totales - 1))
        print("Pieza original:", primera_pieza)
        barco.append(primera_pieza)
        orientacion = random.choice(["N","S","O","E"])
        print("Con orientacion", orientacion)
        fila, columna = primera_pieza
        for i in range(eslora - 1):
            if orientacion == "N":
                fila -= 1
            elif orientacion  == "S":
                fila += 1
            elif orientacion == "O":
                columna -= 1
            elif orientacion == "E":
                columna += 1
            pieza = (fila,columna)
            barco.append(pieza)
        tablero_temp = colocar_barco_plus(tablero, barco)

        if type(tablero_temp) == np.ndarray:
            print("Barco colocado, número de intentos:", contador)
            return tablero_temp
        
        print("Tengo que intentar colocar otro barco")

In [ ]:
PLAN_FLOTA = {4: 1, 3: 2, 2: 3, 1: 4}  # E4×1, E3×2, E2×3, E1×4

def colocar_flota_aleatoria(tablero_inicial, plan=PLAN_FLOTA,
                             reintentos_por_barco=200, verbose=False):
    """
    Coloca todos los barcos indicados en 'plan' sobre 'tablero_inicial'.
    Devuelve (tablero_final, flota) donde flota = { "E<eslora>-<i>": set(coords), ... }.
    """
    tab = tablero_inicial
    flota = {}

    for eslora, cantidad in plan.items():
        for i in range(1, cantidad + 1):
            nombre = f"E{eslora}-{i}"
            colocado = False
            intentos = 0
            while intentos < reintentos_por_barco and not colocado:
                intentos += 1
                res_tab, coords = crea_barco_aleatorio(
                    tab, eslora=eslora, num_intentos=100,
                    verbose=False, retornar_coords=True
                )
                if isinstance(res_tab, np.ndarray) and coords:
                    # éxito
                    tab = res_tab
                    flota[nombre] = set(coords)
                    colocado = True
                    if verbose:
                        print(f"{nombre} colocado en {coords} tras {intentos} intentos.")
                # si no, vuelve a intentar
            if not colocado:
                raise RuntimeError(f"No pude colocar {nombre} (eslora {eslora}).")
    return tab


In [53]:
tab = colocar_flota_aleatoria(tablero)

No puedo poner la pieza (0, 3) porque hay otro barco


In [1]:
tab

NameError: name 'tab' is not defined

In [66]:
def disparo(tablero):
    f, c = map(int, input("Escriba las coordenadas a disparar (fila,columna): ")
                     .replace(" ", "")
                     .split(","))
    print(f)
    print(c)
    if tablero[f,c] == "O":
        tablero[f,c] = "X"
    else:
        tablero[f,c] = "-"
    return tablero
disparo(tab) 

Escriba las coordenadas a disparar (fila,columna):  5,8


5
8


array([[' ', ' ', 'O', 'O', 'O', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'O'],
       ['O', ' ', 'O', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['O', ' ', 'O', ' ', 'O', 'O', 'O', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '-', ' '],
       [' ', ' ', ' ', 'O', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', 'O', ' ', ' ', ' ', ' ', ' ', ' '],
       ['O', ' ', ' ', 'O', 'O', 'O', ' ', ' ', 'O', ' '],
       [' ', ' ', ' ', 'O', ' ', ' ', ' ', ' ', 'O', ' ']], dtype='<U1')

1) **Qué es la clase y qué no**

Clase Tablero: representa el tablero de un jugador (uno para vos, otro para la máquina).
Atributos del objeto Tablero (lo que “vive” dentro de cada jugador):
grid_barcos (np.ndarray 10×10): dónde están sus barcos y los impactos que recibió.
grid_tiros (np.ndarray 10×10): registro de dónde este jugador disparó al rival (impacto o agua).
flota (dict): barcos de este jugador con sus coordenadas (para saber hundidos).
vidas (int): casillas con barco que aún no fueron impactadas.
nombre o jugador_id (str): “Usuario” o “Máquina”.
Constantes (no son atributos del tablero; viven en variables.py):
Dimensiones: BOARD_SIZE = 10
Estados de celda: AGUA = 0, BARCO = 1, IMPACTO = 2, AGUA_DISPARADA = 3
Mapas de impresión: VISUAL_JUGADOR y VISUAL_RIVAL (qué símbolo se imprime para cada estado).

Nota clave: “agua, barco, disparo y al agua, disparo y al barco” no son “atributos del objeto”; son estados posibles de cada celda (constantes) que usás para rellenar los np.ndarray.

2) **¿Hay solo dos “objetivos” (tableros)?**
Correcto: dos instancias de Tablero:
tablero_usuario = Tablero("Usuario")
tablero_maquina = Tablero("Máquina")
Cada una tiene dos grids internos:
grid_barcos (lo oculto del dueño + impactos recibidos).
grid_tiros (lo que ese jugador ve del rival: solo tiros efectuados y su resultado).
Eso permite:
Vos ves tu grid_barcos (porque es tu tablero) y el grid_tiros tuyo (tu “mapa de disparos” al rival).
No ves el grid_barcos del rival (sus barcos ocultos); sólo ves tus impactos/aguas sobre él (tu grid_tiros).

3) **Qué ve cada uno (resumen visual)**
Vista que se imprime	Origen de datos	Qué muestra
“Mi tablero”	mi_tablero.grid_barcos + VISUAL_JUGADOR	Barcos propios, impactos recibidos, aguas disparadas por el rival.
“Tablero enemigo”	mi_tablero.grid_tiros + VISUAL_RIVAL	Solo los tiros que vos hiciste (impacto/agua). Nunca barcos ocultos del rival.
4) **Flujo mínimo de juego (conceptual)**

Iniciar ambos tableros:
tablero_usuario.inicializar_flota()
tablero_maquina.inicializar_flota()
En tu turno:
Elegís coordenada → tablero_usuario.disparar_a(tablero_maquina, f, c)
Internamente: el disparo lee/modifica tablero_maquina.grid_barcos y marca el resultado en tablero_usuario.grid_tiros.
En turno de la máquina:
Coord aleatoria → tablero_maquina.disparar_a(tablero_usuario, f, c)
Impacta tu grid_barcos y marca en grid_tiros de la máquina.
Mostrar en consola:
Tu tablero: imprimir_tablero(tablero_usuario.grid_barcos, VISUAL_JUGADOR)
Enemigo (tu registro de tiros): imprimir_tablero(tablero_usuario.grid_tiros, VISUAL_RIVAL)
Termina cuando tablero_usuario.esta_derrotado() o tablero_maquina.esta_derrotado().

5) **Micro-esqueleto de clase (sólo para fijar ideas)**

No ejecutemos aún nada nuevo; es para que se vea dónde va cada cosa.
class Tablero:
    def __init__(self, nombre, n=BOARD_SIZE):
        self.nombre = nombre
        self.n = n
        self.grid_barcos = np.zeros((n, n), dtype=np.int8)  # tus barcos + impactos recibidos
        self.grid_tiros  = np.zeros((n, n), dtype=np.int8)  # tus tiros al rival
        self.flota = {}     # {"E3-1": {(f,c),...}, ...}
        self.vidas = 0

    def inicializar_flota(self):
        # Usa tu lógica de colocación aleatoria ya hecha
        self.grid_barcos, self.flota = colocar_flota_aleatoria(self.grid_barcos)
        self.vidas = sum(len(c) for c in self.flota.values())

    def recibir_disparo(self, f, c):
        # Si cae en barco, marcar IMPACTO y restar vida; si cae en agua, marcar AGUA_DISPARADA
        ...

    def disparar_a(self, rival, f, c):
        # Llama rival.recibir_disparo, y marca en MI grid_tiros IMPACTO o AGUA_DISPARADA
        ...

    def esta_derrotado(self):
        return self.vidas == 0

6) **Próximo pasito (propuesta)**

Implementamos solo imprimir_tablero y mostrar_tablero_propio / mostrar_tablero_enemigo para que vos veas claramente la diferencia entre lo que ves de tu tablero y lo que ves del rival.
Luego agregamos recibir_disparo y disparar_a con lógica mínima y probamos un par de tiros “a mano” antes de armar el while.

¿Seguimos con la función imprimir_tablero (la versión simple con letras A–J y columnas 1–10) y un par de ejemplos de impresión?


In [ ]:
import numpy as np

LETRAS = "ABCDEFGHIJ" # filas A–J

def imprimir_tablero(tablero_np: np.ndarray, visual_map: dict[int, str], titulo: str = "") -> None: """ Imprime un tablero 10x10 usando letras A–J para filas y 1–10 para columnas. - tablero_np: np.ndarray (10x10) con enteros (estados). - visual_map: dict que mapea estado -> string a imprimir (p.ej. " . ", " B ", " X ", " o "). - titulo: (opcional) título arriba del tablero. """ # Validaciones básicas if tablero_np.shape != (10, 10): raise ValueError(f"Se esperaba un tablero 10x10, pero recibí {tablero_np.shape}.")

# Cabecera (columnas 1–10)
if titulo:
    print(titulo)
encabezado = "    " + " ".join([f"{c:>2}" for c in range(1, 11)])
print(encabezado)

# Filas A–J
for i in range(10):
    fila_letra = LETRAS[i]
    # cada celda se mapea con visual_map; si no está, imprimimos " ? "
    celdas = []
    for j in range(10):
        val = int(tablero_np[i, j])
        celdas.append(visual_map.get(val, " ? "))
    print(f"{fila_letra:>2}  " + "".join(celdas))

In [ ]:
import numpy as np
from variables import BOARD_SIZE, AGUA, BARCO, IMPACTO, AGUA_DISPARADA
from funciones import imprimir_tablero
from crear_flota import colocar_flota_aleatoria  # la función que ya tenés

class Tablero:
    def __init__(self, nombre_jugador):
        self.nombre = nombre_jugador
        self.grid_barcos = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=np.int8)
        self.grid_tiros = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=np.int8)
        self.flota = {}  # acá guardamos las coords de cada barco
        self.vidas = 0

    def inicializar_flota(self):
        """Coloca aleatoriamente todos los barcos de este jugador."""
        self.grid_barcos, self.flota = colocar_flota_aleatoria(self.grid_barcos)
        # contamos cuántas casillas con barco hay en total
        self.vidas = sum(len(coords) for coords in self.flota.values())

    def mostrar_tablero_propio(self):
        """Muestra tu tablero (con tus barcos e impactos)."""
        from variables import VISUAL_JUGADOR
        imprimir_tablero(self.grid_barcos, VISUAL_JUGADOR)

    def mostrar_tablero_enemigo(self):
        """Muestra los disparos que hiciste al enemigo."""
        from variables import VISUAL_RIVAL
        imprimir_tablero(self.grid_tiros, VISUAL_RIVAL)

    def recibir_disparo(self, fila, col):
        """El rival te dispara en (fila, col). Devuelve True si impactó."""
        if self.grid_barcos[fila, col] == BARCO:
            self.grid_barcos[fila, col] = IMPACTO
            self.vidas -= 1
            return True
        elif self.grid_barcos[fila, col] == AGUA:
            self.grid_barcos[fila, col] = AGUA_DISPARADA
            return False

    def disparar_a(self, enemigo, fila, col):
        """Dispara al tablero del enemigo."""
        impacto = enemigo.recibir_disparo(fila, col)
        # Marcamos el resultado en el tablero de tiros
        if impacto:
            self.grid_tiros[fila, col] = IMPACTO
            print("¡Impacto!")
        else:
            self.grid_tiros[fila, col] = AGUA_DISPARADA
            print("Agua.")
        return impacto

    def esta_derrotado(self):
        """Devuelve True si ya no le quedan barcos."""
        return self.vidas == 0


In [1]:
# Constantes del juego Hundir la Flota
# Dimensiones del tablero
DIMENSIONES = 10
# Diccionario con los barcos y sus esloras
BARCOS = {
    'barco_1_a': 1,
    'barco_1_b': 1,
    'barco_1_c': 1,
    'barco_1_d': 1,
    'barco_2_a': 2,
    'barco_2_b': 2,
    'barco_2_c': 2,
    'barco_3_a': 3,
    'barco_3_b': 3,
    'barco_4': 4
}
# Símbolos para representar el tablero
AGUA = ' '              # Agua sin impactar
AGUA_IMPACTADA = 'O'    # Agua donde se disparó
BARCO = '■'             # Barco sin impactar (solo visible en tablero propio)
BARCO_IMPACTADO = 'X'   # Barco impactado
BARCO_OCULTO = ' '      # Barco oculto en tablero enemigo
# Orientaciones posibles para colocar barcos
ORIENTACIONES = ['N', 'S', 'E', 'O']  # Norte, Sur, Este, Oeste
# Total de vidas (suma de todas las esloras)
VIDAS_TOTALES = sum(BARCOS.values())  # 4*1 + 3*2 + 2*3 + 1*4 = 20

In [4]:
import random
random.randint(0, 10)

11